### Wrangle - modules used
- Called functions from my wrangle.py module for the acquire and prepare steps
- Used 'get_connection' function to access data from the Codeup database by using credentionals stored in my 'env.py' file with the required user, host, and password information required for verification in conjucntion with the desired database name to create a unique url.
- Used 'new_zillow_data' function to join
- Etc...

### Wrangle - missing values addressed
- The missing values were addressed by determing if there were any null values
- There were 